In [ ]:
import json
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 1. MODEL BİLGİLERİ
import os
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("token")

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, token=hf_token, device_map="auto", torch_dtype="auto")

qa_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device_map="auto"
)


# 2. VERİYİ YÜKLE
data_path = "/kaggle/input/kbb-test-data/kbb_qa_test.json"
with open(data_path, "r", encoding="utf-8") as f:
    all_data = json.load(f)

# 3. RASTGELE 10 SORU SEÇ 
sample_data = random.sample(all_data, 10)

# 4. SORULARI MODELE SOR
results = []

for idx, item in enumerate(sample_data):
    question = item["question"].strip()
    prompt = (
        f"<s>### Soru:\n{question}\n\n### Cevap:\n"
    )

    print(f"\n[{idx+1}] Soru: {question}")
    outputs = qa_pipeline(prompt)
    # Mistral genellikle tüm promptu tekrarlar, cevabı '### Cevap:' sonrası kısmı alıyoruz:
    model_answer = outputs[0]['generated_text'].split("### Cevap:")[-1].strip()
    doctor_answer = item.get("answer", "").strip()

    print("[Doktor Cevabı]:", doctor_answer)
    print("[Model Cevabı]:", model_answer)

    results.append({
        "topic": item.get("topic", ""),
        "title": item.get("title", ""),
        "question": question,
        "doctor_answer": doctor_answer,
        "model_answer": model_answer,
        "doctorID": item.get("doctorID", "")
    })

# 5. SONUÇLARI KAYDET
with open("mistral_zero_shot_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("\nZero-shot sonuçları 'mistral_zero_shot_results.json' dosyasına kaydedildi.")
